In [1]:
from influxdb_client import InfluxDBClient
import pandas as pd
import torch
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from data.dataset_loader import add_contextual_features
from models.lstm import LSTMModel
from models.hybrid import HybridLSTMTransformerModel
from models.transformer import TransformerModel
from pprint import pprint

In [2]:
from utils.train import evaluate_model

features = ['airquality', 'light', 'hour_of_day', 'is_weekend', 'airquality_delta', 'airquality_trend']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def create_sequences(X, y, seq_len=30):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_len + 1):
        X_seq.append(X[i:i+seq_len])
        y_seq.append(y[i + seq_len - 1])  # label at last time step
    return np.array(X_seq), np.array(y_seq)

def evaluate_on_manual_df(manual_df, features, model_paths, seq_len=30, batch_size=64, device='cpu'):
    results = {}

    # Drop missing rows
    df = manual_df.dropna(subset=features + ['occupancy'])
    X = df[features].values.astype(float)
    y = df['occupancy'].values.astype(float)

    # Create sequences
    X_seq, y_seq = create_sequences(X, y, seq_len=seq_len)
    X_tensor = torch.tensor(X_seq, dtype=torch.float32).to(device)
    y_tensor = torch.tensor(y_seq, dtype=torch.float32).to(device)

    # Create DataLoader
    dataset = TensorDataset(X_tensor, y_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    # Load models from provided dictionary and evaluate
    for model_name, checkpoint_path in model_paths.items():
        model = load_model(
        model_name=model_name,
        input_dim=len(features),
        checkpoint_path=checkpoint_path,
        device=device
        )
        result = evaluate_model(model, dataloader, loss_fn=torch.nn.BCEWithLogitsLoss(), device=device, split_name="manual")
        results[model_name] = result

    return results

def load_model(model_name, checkpoint_path, device, input_dim=len(features), **kwargs):
    model = None

    if model_name == 'lstm':
        model = LSTMModel(input_dim, hidden_dim=64, num_layers=2, dropout=0.2)
        model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    elif model_name == 'transformer':
        model = TransformerModel(input_dim, d_model=64, nhead=4, num_encoder_layers=2, dropout=0.2)
        model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    elif model_name == 'hybrid':
        lstm_model = LSTMModel(input_dim, hidden_dim=64, num_layers=2, dropout=0.2)
        transformer_model = TransformerModel(input_dim, d_model=64, nhead=4, num_encoder_layers=2, dropout=0.2)

        model = HybridLSTMTransformerModel(lstm_model, transformer_model)
        model.load_state_dict(torch.load(checkpoint_path, map_location=device))

    model = model.to(device)
    model.eval()
    return model

## All Rooms

In [29]:
client = InfluxDBClient(
    url='http://localhost:8086',
    token='VvreOjZoLYmSZKbpufKm0boJlNfifTSToscteblxZwEetIRMP3IGdUu-IMqRkHNhKy9_o5hfDX56IXEtcRifhw==',
    org='miguel_master_thesis',
    bucket='lab42_sensor_data_manual',
    timeout=3_600_00
)
query_api = client.query_api()

query = f'''
from(bucket: "lab42_sensor_data_manual")
  |> range(start: 2025-01-01T00:00:00Z, stop: 2025-05-28T23:59:59Z)
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
  |> keep(columns: ["_time", "room_number", "temperature", "airquality", "light", "daylight", "capacity", "occupancy"])
'''

manual_df = query_api.query_data_frame(query, org='miguel_master_thesis')

# Combine if multiple tables
if isinstance(manual_df, list):
    manual_df = pd.concat(manual_df, ignore_index=True)

manual_df['_time'] = pd.to_datetime(manual_df['_time'])
manual_df = add_contextual_features(manual_df, normalize=True)

Dropped 0 rows due to NaNs.


In [4]:
model_paths = {
    'lstm': 'checkpoints_and_metrics/lstm_track2.pt',
    'transformer': 'checkpoints_and_metrics/transformer_track2.pt',
    'hybrid': 'checkpoints_and_metrics/hybrid_track2.pt'
}

results = evaluate_on_manual_df(manual_df, features, model_paths, seq_len=30, batch_size=64, device=device)

In [5]:
pprint(results) # Track2 All Rooms

{'hybrid': {'accuracy': 0.6459731179293423,
            'aur_roc': np.float64(0.6728206499077101),
            'confusion_matrix': [[6440, 10212], [6092, 23309]],
            'f1': 0.7408855408283271,
            'loss': 0.7634715053530979,
            'precision': 0.6953551505026699,
            'recall': 0.7927961633958028},
 'lstm': {'accuracy': 0.6580027359781122,
          'aur_roc': np.float64(0.6698751508245389),
          'confusion_matrix': [[5791, 10861], [4889, 24512]],
          'f1': 0.7568468830086146,
          'loss': 0.754405981542853,
          'precision': 0.6929579057473214,
          'recall': 0.8337131390088772},
 'transformer': {'accuracy': 0.6550930449699259,
                 'aur_roc': np.float64(0.6704236638551098),
                 'confusion_matrix': [[7337, 9315], [6569, 22832]],
                 'f1': 0.7419250016247482,
                 'loss': 0.7282477437246901,
                 'precision': 0.7102373471863627,
                 'recall': 0.7765722254345

In [7]:
from tabulate import tabulate

headers = ["model_track2_allrooms", "accuracy", "f1", "precision", "recall", "aur_roc", "loss"]
rows = []

for model, metrics in results.items():
    rows.append([
        model,
        round(metrics["accuracy"], 4),
        round(metrics["f1"], 4),
        round(metrics["precision"], 4),
        round(metrics["recall"], 4),
        round(float(metrics["aur_roc"]), 4),
        round(metrics["loss"], 4)
    ])

print(tabulate(rows, headers=headers, tablefmt="github"))

| model_track2_allrooms   |   accuracy |     f1 |   precision |   recall |   aur_roc |   loss |
|-------------------------|------------|--------|-------------|----------|-----------|--------|
| lstm                    |     0.658  | 0.7568 |      0.693  |   0.8337 |    0.6699 | 0.7544 |
| transformer             |     0.6551 | 0.7419 |      0.7102 |   0.7766 |    0.6704 | 0.7282 |
| hybrid                  |     0.646  | 0.7409 |      0.6954 |   0.7928 |    0.6728 | 0.7635 |


## Singular Rooms

### Room_11

In [19]:
client = InfluxDBClient(
    url='http://localhost:8086',
    token='VvreOjZoLYmSZKbpufKm0boJlNfifTSToscteblxZwEetIRMP3IGdUu-IMqRkHNhKy9_o5hfDX56IXEtcRifhw==',
    org='miguel_master_thesis',
    bucket='lab42_sensor_data_manual',
    timeout=3_600_00
)
query_api = client.query_api()

query = f'''
from(bucket: "lab42_sensor_data_manual")
  |> range(start: 2025-01-01T00:00:00Z, stop: 2025-05-28T23:59:59Z)
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
  |> keep(columns: ["_time", "room_number", "temperature", "airquality", "light", "daylight", "capacity", "occupancy"])
'''

manual_df = query_api.query_data_frame(query, org='miguel_master_thesis')

# Combine if multiple tables
if isinstance(manual_df, list):
    manual_df = pd.concat(manual_df, ignore_index=True)

manual_df['_time'] = pd.to_datetime(manual_df['_time'])
manual_df = manual_df[manual_df['room_number'] == 'Room_11']
manual_df = add_contextual_features(manual_df, normalize=True)

Dropped 0 rows due to NaNs.


In [20]:
model_paths = {
    'lstm': 'checkpoints_and_metrics/lstm_track2_room11.pt',
    'transformer': 'checkpoints_and_metrics/transformer_track2_room11.pt',
    'hybrid': 'checkpoints_and_metrics/hybrid_track2_room11.pt'
}

results = evaluate_on_manual_df(manual_df, features, model_paths, seq_len=30, batch_size=64, device=device)

In [21]:
pprint(results)

{'hybrid': {'accuracy': 0.6493301812450749,
            'aur_roc': np.float64(0.8581021245480767),
            'confusion_matrix': [[439, 39], [851, 1209]],
            'f1': 0.7309552599758162,
            'loss': 1.086754083416679,
            'precision': 0.96875,
            'recall': 0.5868932038834952},
 'lstm': {'accuracy': 0.6382978723404256,
          'aur_roc': np.float64(0.8666155502295163),
          'confusion_matrix': [[435, 43], [875, 1185]],
          'f1': 0.7208029197080292,
          'loss': 0.6376184248838399,
          'precision': 0.9649837133550488,
          'recall': 0.5752427184466019},
 'transformer': {'accuracy': 0.6469661150512215,
                 'aur_roc': np.float64(0.8451842222854125),
                 'confusion_matrix': [[438, 40], [856, 1204]],
                 'f1': 0.7288135593220338,
                 'loss': 1.3805367771201418,
                 'precision': 0.9678456591639871,
                 'recall': 0.5844660194174758}}


In [22]:
from tabulate import tabulate

headers = ["model_track2_room11", "accuracy", "f1", "precision", "recall", "aur_roc", "loss"]
rows = []

for model, metrics in results.items():
    rows.append([
        model,
        round(metrics["accuracy"], 4),
        round(metrics["f1"], 4),
        round(metrics["precision"], 4),
        round(metrics["recall"], 4),
        round(float(metrics["aur_roc"]), 4),
        round(metrics["loss"], 4)
    ])

print(tabulate(rows, headers=headers, tablefmt="github"))

| model_track2_l110   |   accuracy |     f1 |   precision |   recall |   aur_roc |   loss |
|---------------------|------------|--------|-------------|----------|-----------|--------|
| lstm                |     0.6383 | 0.7208 |      0.965  |   0.5752 |    0.8666 | 0.6376 |
| transformer         |     0.647  | 0.7288 |      0.9678 |   0.5845 |    0.8452 | 1.3805 |
| hybrid              |     0.6493 | 0.731  |      0.9688 |   0.5869 |    0.8581 | 1.0868 |


### Room_02

In [24]:
client = InfluxDBClient(
    url='http://localhost:8086',
    token='VvreOjZoLYmSZKbpufKm0boJlNfifTSToscteblxZwEetIRMP3IGdUu-IMqRkHNhKy9_o5hfDX56IXEtcRifhw==',
    org='miguel_master_thesis',
    bucket='lab42_sensor_data_manual',
    timeout=3_600_00
)
query_api = client.query_api()

query = f'''
from(bucket: "lab42_sensor_data_manual")
  |> range(start: 2025-01-01T00:00:00Z, stop: 2025-05-28T23:59:59Z)
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
  |> keep(columns: ["_time", "room_number", "temperature", "airquality", "light", "daylight", "capacity", "occupancy"])
'''

manual_df = query_api.query_data_frame(query, org='miguel_master_thesis')

# Combine if multiple tables
if isinstance(manual_df, list):
    manual_df = pd.concat(manual_df, ignore_index=True)

manual_df['_time'] = pd.to_datetime(manual_df['_time'])
manual_df = manual_df[manual_df['room_number'] == 'Room_02']
manual_df = add_contextual_features(manual_df, normalize=True)

Dropped 0 rows due to NaNs.


In [25]:
model_paths = {
    'lstm': 'checkpoints_and_metrics/lstm_track2_room02.pt',
    'transformer': 'checkpoints_and_metrics/transformer_track2_room02.pt',
    'hybrid': 'checkpoints_and_metrics/hybrid_track2_room02.pt'
}

results = evaluate_on_manual_df(manual_df, features, model_paths, seq_len=30, batch_size=64, device=device)

In [26]:
pprint(results)

{'hybrid': {'accuracy': 0.8526398739164697,
            'aur_roc': np.float64(0.6617406734896005),
            'confusion_matrix': [[9, 199], [175, 2155]],
            'f1': 0.9201537147736977,
            'loss': 0.5528194910541202,
            'precision': 0.9154630416312659,
            'recall': 0.924892703862661},
 'lstm': {'accuracy': 0.8353033884948778,
          'aur_roc': np.float64(0.6440017332452955),
          'confusion_matrix': [[35, 173], [245, 2085]],
          'f1': 0.9088927637314734,
          'loss': 0.5978494608825713,
          'precision': 0.9233835252435784,
          'recall': 0.8948497854077253},
 'transformer': {'accuracy': 0.8498817966903073,
                 'aur_roc': np.float64(0.6274801914823374),
                 'confusion_matrix': [[15, 193], [188, 2142]],
                 'f1': 0.9183279742765273,
                 'loss': 0.4629684198531777,
                 'precision': 0.9173447537473234,
                 'recall': 0.91931330472103}}


In [27]:
from tabulate import tabulate

headers = ["model_track2_room02", "accuracy", "f1", "precision", "recall", "aur_roc", "loss"]
rows = []

for model, metrics in results.items():
    rows.append([
        model,
        round(metrics["accuracy"], 4),
        round(metrics["f1"], 4),
        round(metrics["precision"], 4),
        round(metrics["recall"], 4),
        round(float(metrics["aur_roc"]), 4),
        round(metrics["loss"], 4)
    ])

print(tabulate(rows, headers=headers, tablefmt="github"))

| model_track2_l009   |   accuracy |     f1 |   precision |   recall |   aur_roc |   loss |
|---------------------|------------|--------|-------------|----------|-----------|--------|
| lstm                |     0.8353 | 0.9089 |      0.9234 |   0.8948 |    0.644  | 0.5978 |
| transformer         |     0.8499 | 0.9183 |      0.9173 |   0.9193 |    0.6275 | 0.463  |
| hybrid              |     0.8526 | 0.9202 |      0.9155 |   0.9249 |    0.6617 | 0.5528 |


### Room_06

In [30]:
client = InfluxDBClient(
    url='http://localhost:8086',
    token='VvreOjZoLYmSZKbpufKm0boJlNfifTSToscteblxZwEetIRMP3IGdUu-IMqRkHNhKy9_o5hfDX56IXEtcRifhw==',
    org='miguel_master_thesis',
    bucket='lab42_sensor_data_manual',
    timeout=3_600_00
)
query_api = client.query_api()

query = f'''
from(bucket: "lab42_sensor_data_manual")
  |> range(start: 2025-01-01T00:00:00Z, stop: 2025-05-28T23:59:59Z)
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
  |> keep(columns: ["_time", "room_number", "temperature", "airquality", "light", "daylight", "capacity", "occupancy"])
'''

manual_df = query_api.query_data_frame(query, org='miguel_master_thesis')

# Combine if multiple tables
if isinstance(manual_df, list):
    manual_df = pd.concat(manual_df, ignore_index=True)

manual_df['_time'] = pd.to_datetime(manual_df['_time'])
manual_df = manual_df[manual_df['room_number'] == 'Room_06']
manual_df = add_contextual_features(manual_df, normalize=True)

Dropped 0 rows due to NaNs.


In [31]:
model_paths = {
    'lstm': 'checkpoints_and_metrics/lstm_track2_room06.pt',
    'transformer': 'checkpoints_and_metrics/transformer_track2_room06.pt',
    'hybrid': 'checkpoints_and_metrics/hybrid_track2_room06.pt'
}

results = evaluate_on_manual_df(manual_df, features, model_paths, seq_len=30, batch_size=64, device=device)

In [32]:
pprint(results)

{'hybrid': {'accuracy': 0.709105242412298,
            'aur_roc': np.float64(0.864251848136338),
            'confusion_matrix': [[283, 107], [631, 1516]],
            'f1': 0.8042440318302387,
            'loss': 0.5335944560080179,
            'precision': 0.9340727048675292,
            'recall': 0.7061015370284117},
 'lstm': {'accuracy': 0.7000394166338195,
          'aur_roc': np.float64(0.8606953053157058),
          'confusion_matrix': [[284, 106], [655, 1492]],
          'f1': 0.7967957276368491,
          'loss': 0.5689420983436776,
          'precision': 0.9336670838548186,
          'recall': 0.6949231485794132},
 'transformer': {'accuracy': 0.7323610563657864,
                 'aur_roc': np.float64(0.8698458194499182),
                 'confusion_matrix': [[287, 103], [576, 1571]],
                 'f1': 0.8222978277937713,
                 'loss': 0.6342906895413762,
                 'precision': 0.9384707287933094,
                 'recall': 0.7317186772240335}}


In [33]:
from tabulate import tabulate

headers = ["model_track2_room06", "accuracy", "f1", "precision", "recall", "aur_roc", "loss"]
rows = []

for model, metrics in results.items():
    rows.append([
        model,
        round(metrics["accuracy"], 4),
        round(metrics["f1"], 4),
        round(metrics["precision"], 4),
        round(metrics["recall"], 4),
        round(float(metrics["aur_roc"]), 4),
        round(metrics["loss"], 4)
    ])

print(tabulate(rows, headers=headers, tablefmt="github"))

| model_track2_l101   |   accuracy |     f1 |   precision |   recall |   aur_roc |   loss |
|---------------------|------------|--------|-------------|----------|-----------|--------|
| lstm                |     0.7    | 0.7968 |      0.9337 |   0.6949 |    0.8607 | 0.5689 |
| transformer         |     0.7324 | 0.8223 |      0.9385 |   0.7317 |    0.8698 | 0.6343 |
| hybrid              |     0.7091 | 0.8042 |      0.9341 |   0.7061 |    0.8643 | 0.5336 |
